In [1]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from pathlib import Path
import os

In [2]:
model_name = "distilgpt2"
model_save_path = Path(os.getcwd()) / 'models'
weights_path = f'{model_save_path}/gpt2_all_data_model.pt'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(weights_path)
print(device)

/home/isachansson/DML-project/notebooks/models/gpt2_all_data_model.pt
cuda


In [3]:
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained(model_name)
model.load_state_dict(torch.load(weights_path, map_location=device))
model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [5]:
prompt = "Genre: Heavy Metal\n\n"
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

with torch.inference_mode():
    outputs = model.generate(
        input_ids,
        do_sample=True,
        max_length=512,
        top_p=0.9,
        top_k=50,
        temperature=0.9,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        num_return_sequences=2,
    )
for i, output in enumerate(outputs, 1):
    text = tokenizer.decode(output, skip_special_tokens=True)
    print(f"\n{'='*60}")
    print(f"SAMPLE {i}")
    print(f"{'='*60}")
    print(text)

print(text)


SAMPLE 1
Genre: Heavy Metal


I need a new face I know it's coming
It's hard to see you are standing in the rain
As my mind turns to stone
I want you to stay by my side

My life's not a lie
And I don't wanna cry
But your love has left me feeling alone
The way I was when I was a boy
You know that I am so lonely

So, tell me if you still care
Tell me now what would I do?
Will I be living again?
What would I be doing with you?
When everything changes like the day
We have to fight each other out

My life's not a lie
And I don't wanna cry
But your love has left me feeling alone
The way I was when I was a boy
You know that I am so lonely

So, tell me if you still care
Tell me now what would I do?
Will I be living again?
What would I be doing with you?
When everything changes like the day
We have to fight each other out

Can't say goodbye
'Cause I will never change
If we go this far
There is no place for us
This will only be

So let me tell you all the things you've said
Don't say goodbye
No